In [ ]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,Dropout
# from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import scipy  #Used to upsample our image
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.utils.class_weight import compute_class_weight
import os
import cv2
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Funciones auxiliares

In [ ]:
def print_layers(model):
  for layer in model.layers:
    print(layer.name,layer.trainable)

def plot_history(history):
  #plot the training and validation accuracy and loss at each epoch
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(loss) + 1)
  plt.plot(epochs, loss, 'y', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
  
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  plt.plot(epochs, acc, 'y', label='Training acc')
  plt.plot(epochs, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

#Create model

In [ ]:
def get_model(input_shape = (224,224,3)):
    conv_base = ResNet50(weights = None,#weights='imagenet',
                      include_top=False,
                      input_shape=input_shape)
    x = conv_base.output
    x = GlobalAveragePooling2D()(x)
    # x = Dense(2048, activation="relu")(x) 
    # x = Dropout(0.5)(x)
    x = Dense(2, activation="softmax")(x)
    
    model = Model(conv_base.input, x)
    model.compile(loss='categorical_crossentropy',
                optimizer= optimizers.Adam(learning_rate=1e-5),
                metrics=['accuracy'])# internamente detecta la funcion los y usa categorical_accuracy al indicar accuracy
    return model

model = get_model(input_shape = (224,224,3))
print(model.summary())

# Train the model

In [ ]:
print_layers(model)

##Create dataset

In [ ]:
directory = '/content/drive/MyDrive/FIRE DETECTION/data/Fire Dataset PCD'
batch_size = 16
target_size = model.layers[0].output_shape[0][1:3]#(224,224)

train_datagen = ImageDataGenerator(
    # rescale=1./255,
    preprocessing_function=preprocess_input,
    rotation_range=60,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(
    #rescale=1./255,
    preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory(
    directory=directory + '/Train',
    batch_size=batch_size,
    shuffle=True,
    target_size= target_size,
    save_to_dir ='/content/drive/MyDrive/prueba')

validation_generator=val_datagen.flow_from_directory(
    directory=directory + '/Test',
    batch_size=batch_size,
    shuffle = False,
    target_size= target_size)

In [ ]:
cls_train = train_generator.classes #labels en train
class_names = list(validation_generator.class_indices.keys())
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(cls_train),
                                    y=cls_train)
class_weight = {i : class_weight[i] for i in range(len(class_weight))}
print(class_weight)
print(class_names)

##- Transfer learning:

In [ ]:
def setup_to_transfer_learning(model):
  """Freeze all layers and compile the model"""
  for layer in model.layers[:-4]:
    layer.trainable = False
  for layer in model.layers[-4:]:
    layer.trainable = True
  model.compile(optimizer='rmsprop', #optimizer=Adam(learning_rate=1e-5),   
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
  print("model compiled for transfer")
  # print(model.summary())  

In [ ]:
setup_to_transfer_learning(model)
print_layers(model)

In [ ]:
# Include the epoch in the file name (uses `str.format`)
ckpt_dir = '/content/drive/MyDrive/FIRE DETECTION/models/ResNet transfer learning'
checkpoint_path = os.path.join(ckpt_dir,"weights-improvement-{epoch:02d}-{val_accuracy:.2f}.ckpt")

# Create a callback that saves the model's weights every epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=True)

# # Save the weights using the `checkpoint_path` format
# model.save_weights(checkpoint_path.format(epoch=0))

history = model.fit(train_generator,
                    epochs=5,
                    callbacks=[cp_callback],
                    steps_per_epoch = train_generator.samples // batch_size,
                    class_weight = class_weight,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.samples // batch_size,
                    verbose = 1)

plot_history(history)

##- Fine-tuning:

In [ ]:
def setup_to_finetune(model):
  for layer in model.layers[-4:]:
    layer.trainable = True
  for layer in model.layers[:-4]:
    if ('conv5' not in layer.name):# and ('block4' not in layer.name):
      layer.trainable = False 
    else:
      layer.trainable = True
  model.compile(optimizer=Adam(learning_rate=1e-7),   
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  print("model compiled")
  # print(model.summary())
  # model.compile(loss='categorical_crossentropy',
  #                 optimizer= optimizers.Adam(learning_rate=1e-5),
  #                 metrics=['accuracy'])


In [ ]:
setup_to_finetune(model)
print_layers(model)

In [ ]:
# Include the epoch in the file name (uses `str.format`)
ckpt_dir = '/content/drive/MyDrive/FIRE DETECTION/models/sctach resnet'
checkpoint_path = os.path.join(ckpt_dir,"weights-improvement-{epoch:02d}-{val_accuracy:.2f}.ckpt")

# Create a callback that saves the model's weights every epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=True)

history = model.fit(train_generator,
                    epochs=50,
                    callbacks=[cp_callback],
                    steps_per_epoch = train_generator.samples // batch_size,
                    class_weight = class_weight,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.samples // batch_size,
                    verbose = 1)

plot_history(history)

##Evalua modelo

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator, steps= validation_generator.samples // batch_size, verbose=1)
print('test acc:', test_acc)

#Cargar modelo

In [ ]:
model_path = '/content/drive/MyDrive/FIRE DETECTION/models/sctach resnet'
os.listdir(model_path)

In [ ]:
#Cargar modelo dado el path
def load_model(model_path,model_name):
  model = get_model()
  model.load_weights(os.path.join(model_path,model_name))
  return model

In [ ]:
#Cargar ultimo modelo creado
def load_last_model(model_path):
  latest = tf.train.latest_checkpoint(model_path)
  print('last: ',latest)
  # Create a new model instance
  model = get_model()
  model.load_weights(latest)
  return model

In [ ]:
a = load_model('/content/drive/MyDrive/FIRE DETECTION/models/sctach resnet','weights-improvement-10-0.83.ckpt')

Evalua modelo cargado

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator, steps= validation_generator.samples // batch_size, verbose=1)
print('test acc:', test_acc)

#Predict

In [ ]:
idx_out_con_layer = -5 # -6 :SALIDA DE conv5_block3_add (ANTES de relu), o -5, que es la salida de relu y la entrada a global average pooling
class_names = ['Fuego','No fuego']
last_layer_weights = model.layers[-1].get_weights()[0] #Get weights for all classes from the prediction layer

def process_input_image(image_path):
  # loads RGB image as PIL.Image.Image type
  img = load_img(image_path, target_size=(224, 224))
  # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
  x = img_to_array(img)
  # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
  x = np.expand_dims(x, axis=0)
  # convert RGB -> BGR, subtract mean ImageNet pixel, and return 4D tensor
  return preprocess_input(x)

def compute_heatmaps(model,image_path,last_layer_weights):
  #Get output from the last conv. layer and predictions
  resnet = Model(inputs = model.input, outputs = (model.output,model.layers[idx_out_con_layer].output))
  input = process_input_image(image_path)
  pred,last_conv_output = resnet.predict(input)
  pred_class = np.argmax(pred)

  last_conv_output = np.squeeze(last_conv_output) # (7,7,2048) -> (1,7,7,2048)
  last_layer_weights_for_pred = last_layer_weights[:, pred_class]  #Get weights for the predicted class.

  #Upsample/resize the last conv. output to same size as original image
  h = int(input.shape[1]/last_conv_output.shape[0])
  w = int(input.shape[2]/last_conv_output.shape[1])
  upsampled_last_conv_output = scipy.ndimage.zoom(last_conv_output, (h, w, 1), order=1)
  heat_map = np.dot(upsampled_last_conv_output.reshape((input.shape[1]*input.shape[2], 2048)), 
                last_layer_weights_for_pred).reshape(input.shape[1],input.shape[2])
  return heat_map,pred,pred_class

  # #Detect peaks (hot spots) in the heat map. We will set it to detect maximum 5 peaks.
  # #with rel threshold of 0.5 (compared to the max peak). 
  # peak_coords = peak_local_max(heat_map, num_peaks=10, threshold_rel=0.5, min_distance=10) 
  # plt.imshow(img_aux)
  # plt.imshow(heat_map, cmap='jet', alpha=0.70)
  # for i in range(0,peak_coords.shape[0]):
  #     y = peak_coords[i,0]
  #     x = peak_coords[i,1]
  #     plt.gca().add_patch(Rectangle((x-25, y-25), 50,50,linewidth=1,edgecolor='r',facecolor='none'))


def predict_and_plot(model,image_path,last_layer_weights,ax1, ax2):
  # load image, convert BGR --> RGB, resize image to 224 x 224,
  img = cv2.imread(image_path, 1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = Image.fromarray(img, 'RGB')
  img = img.resize((224, 224))
  ax1.imshow(img, alpha=1)   # alpha is opacity parameter
  # plot image
  ax2.imshow(img, alpha=0.5)
  # get class activation map
  heat_map, pred,pred_class = compute_heatmaps(model,image_path,last_layer_weights)
  # plot class activation map
  ax2.imshow(heat_map, cmap='jet', alpha=0.5)             # overlay heat map from class activation mapping
  title = class_names[pred_class] + ':    ' + str(pred[0][pred_class])
  ax2.set_title(title)

def plot_prediction(file_path,last_layer_weights):
  fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True, figsize=plt.figaspect(0.5))
  predict_and_plot(model,file_path,last_layer_weights,ax1,ax2)
  plt.show()




In [ ]:
plot_prediction('/content/drive/MyDrive/FIRE DETECTION/data/Fire Dataset PCD/Test/Fire/image_23.jpg',last_layer_weights)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True, figsize=plt.figaspect(0.5))
predict_and_plot(model,'/content/drive/MyDrive/FIRE DETECTION/data/Fire Dataset PCD/Test/Fire/image_23.jpg',last_layer_weights,ax1,ax2)
plt.show()

In [ ]:
predict_path = '/content/drive/MyDrive/FIRE DETECTION/data/Fire Dataset PCD/Train/Fire'

class_names = ['Fuefo','No fuego']
last_layer_weights = model.layers[-1].get_weights()[0] #Get weights for all classes from the prediction layer
for filename in os.listdir(predict_path)[15:20]:
  filename = os.path.join(predict_path,filename)
  plot_prediction(filename,last_layer_weights)
